In [1]:
# Imports

import pandas as pd
import numpy as np
import re
from Scripts.normalize_data import getTrueLabelsDataframe, getCleanDataframe, getCleanAiDataframe, apply_normalizations
import Scripts.normalize_data as n
from Scripts.functions import reder, bench_it
from pathlib import Path
import os


In [2]:
df_ground_truth = getTrueLabelsDataframe()

## Loading Data to analyse


In [3]:
#loading dataframes
bandit = getCleanDataframe('C:/Users/levim/projetos/results_mk2/formatted_bandit.json').rename(columns={'labels': 'bandit'})
semgrep = getCleanDataframe('C:/Users/levim/projetos/results_mk2/formatted_semgrep.json').rename(columns={'labels': 'semgrep'})
ai_bandit = getCleanAiDataframe('C:/Users/levim/projetos/results_mk2/ai_results_bandit.json').rename(columns={'ai_predictions': 'ai_bandit'})
ai_semgrep = getCleanAiDataframe('C:/Users/levim/projetos/results_mk2/ai_results_semgrep.json').rename(columns={'ai_predictions': 'ai_semgrep'})

#Merging all of them into a single one containing the ground truths
labels = (
    df_ground_truth
    .merge(bandit[['filename', 'bandit']], on='filename', how='left')
    .merge(semgrep[['filename', 'semgrep']], on='filename', how='left')
    .merge(ai_bandit[['filename', 'ai_bandit']], on='filename', how='left')
    .merge(ai_semgrep[['filename', 'ai_semgrep']], on='filename', how='left')
)

#Applying normalizations on the columns
labels = apply_normalizations(labels, 'bandit')
labels = apply_normalizations(labels, 'semgrep')
labels = apply_normalizations(labels, 'ai_bandit')
labels = apply_normalizations(labels, 'ai_semgrep')

def remove_spaces(text):
    return re.sub(r'(\d) +\(', r'\1(', text)

labels['true_label'] = labels['true_label'].apply(remove_spaces)


## Evaluating the results




In [16]:
#Generating list of predictions
banditPreds = reder(labels, 'bandit')
semgrepPreds = reder(labels, 'semgrep')
aiBanditPreds = reder(labels, 'ai_bandit')
aiSemgrepPreds = reder(labels, 'ai_semgrep')

#Generating list of expected values
exploded_labels = labels['true_label'].str.split(',').explode()
dist_labels = (exploded_labels.index.astype(str) + '-' + exploded_labels).values

teste = {
    'bandit': banditPreds,
    'semgrep': semgrepPreds,
    'ai_bandit': aiBanditPreds,
    'ai_semgrep': aiSemgrepPreds
}


In [19]:
rows = []

for tool, preds in teste.items():
    precision, recall, f1 = bench_it(dist_labels, preds)
    
    rows.append({
        'Tool': tool,
        'Precision': precision,
        'Recall': recall,
        'F1 Score': f1
    })

report_df = pd.DataFrame(rows)

report_df

,Tool,Precision,Recall,F1 Score
0,bandit,0.285714,0.057803,0.096154
1,semgrep,0.677778,0.117534,0.200328
2,ai_bandit,0.328358,0.042389,0.075085
3,ai_semgrep,0.389610,0.057803,0.100671
